# dlt (Deep Learning Tools) パッケージの使い方

この記事は、[Kerasを用いたCIFAR-10チュートリアル](https://qiita.com/hiroyuki827/items/8c59cb26844b6db46f06)で用いるdltパッケージの使い方をまとめたものです。

English version is [HERE](../dltパッケージの使い方/How_to_use_dlt_package/How_to_use_dlt_package.html)

## 誰が作ったか?

もともとこのパッケージを作ったのは、RWTH Aachen工科大学の[David Waltz](https://github.com/DavidWalz/dlipr)氏です。私は彼の「Deep Learning in Physics Research」という講義を取っており、その講義を通してDeep Learningを学びました。この講義で使われていたdltは、大学のクラスター上で動くものでしたが、私は個人のコンピュータでも動くように修正し、`dlt` (Deep Learning Tools)として、ここで公開することにしました。

#### このノートについて



この記事では、それらをどう使うかを紹介していきます。以下では、**[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist/blob/master/README.ja.md)** を例に扱ってみます。ここでは詳しいディープラーニングの内容は解説しませんが、dltを使えばどんな結果を得ることができるかを知っていただくことを目的としているので、ディープラーニングについて知らなくても構いません。少々理解できないところがあっても、後から理解されることをおすすめします。

- `dlt`を使うところでは、★マークを使っています。

## どう使えばよいか?

### 準備

まず、ライブラリのバージョンを示しておきます。

In [1]:
import numpy
numpy.__version__

'1.13.3'

In [2]:
import matplotlib.pyplot
matplotlib.__version__

'2.0.2'

In [3]:
import tensorflow
tensorflow.__version__

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

In [4]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.2'

- dltパッケージを使うために、

```
pip install dlt
```
を実行してください。

### ディープラーニング - Fashion-MNIST

基本的には[MNISTのサンプルコード](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py)を参考にします。

In [5]:
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os 
import numpy as np
import dlt

#### ★ データセットの読み込み

Fashion MNISTのデータセットを読み込むメソッドは以下のようにして読み込みます。

In [6]:
data = dlt.fashion_mnist.load_data()

- dltでは以下のデータセットを用意しています: `CIFAR-10`, `CIFAR-100`, `MNIST`, `FASHION-MNIST`
- 各ファイルに上記の`load_data`に相当するメソッドが定義されています。

各データには以下のようにアクセスできます。Fashion MNISTはMNISTと同じデータ量（フォーマット）で提供されているので、MNISTと同じように扱えます。

##### 訓練データ

In [7]:
X_train = data.train_images
y_train = data.train_labels

In [8]:
X_train.shape

(60000, 28, 28)

In [9]:
y_train.shape

(60000,)

##### テストデータ

In [10]:
X_test = data.test_images
y_test = data.test_labels

X_test.shape

(10000, 28, 28)

In [11]:
y_test.shape

(10000,)

★また、各ターゲットとなるデータセット`y_train`, `y_test`に、正解ラベルがどういう分布で含まれているかは以下のようにしてチェックできます。代表的なデータセットはラベルの偏りがないように、それぞれのラベルに対して同数の学習データが用意されています。自分で画像を持ってきて用意する際も、なるべくこのようにすることが理想的です。

In [12]:
dlt.utils.plot_distribution_data(Y=data.train_labels, #正解ラベルのデータセット
                                 dataset_name='y_train', # そのデータセットの名前
                                 classes=data.classes, # ラベル
                                 fname='dist_train.png' # 出力するファイルパス
                                )

Mean Value: 6000
Median Value: 6000.0
Variance: 0
Standard Deviation: 0.0


<img src="dist_train.png">

In [13]:
dlt.utils.plot_distribution_data(Y=data.test_labels, 
                                 dataset_name='y_test', 
                                 classes=data.classes, 
                                 fname='dist_test.png')

Mean Value: 6000
Median Value: 6000.0
Variance: 0
Standard Deviation: 0.0


<img src="dist_test.png">

#### ★サンプル画像の表示

dltにはサンプル画像を表示するメソッドも用意されています。どういう画像が学習画像となっているかわかりますね。

In [14]:
dlt.utils.plot_examples(data=data, 
                          num_examples=5, # 縦に何個表示するか (横はカテゴリーと一致)
                         fname='fashion_mnist_examples.png' # ファイルパス
                       )

<img src="fashion_mnist_examples.png">

In [15]:
X_train = X_train.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
              
fit = model.fit(X_train, y_train,
          batch_size=128,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 289s 5ms/step - loss: 0.6023 - acc: 0.7908 - val_loss: 0.3954 - val_acc: 0.8592
Epoch 2/12
60000/60000 [==============================] - 350s 6ms/step - loss: 0.3897 - acc: 0.8630 - val_loss: 0.3380 - val_acc: 0.8793
Epoch 3/12
60000/60000 [==============================] - 287s 5ms/step - loss: 0.3415 - acc: 0.8805 - val_loss: 0.3063 - val_acc: 0.8903
Epoch 4/12
60000/60000 [==============================] - 306s 5ms/step - loss: 0.3150 - acc: 0.8892 - val_loss: 0.2884 - val_acc: 0.8974
Epoch 5/12
60000/60000 [==============================] - 310s 5ms/step - loss: 0.2922 - acc: 0.8971 - val_loss: 0.2751 - val_acc: 0.9004
Epoch 6/12
60000/60000 [==============================] - 308s 5ms/step - loss: 0.2746 - acc: 0.9029 - val_loss: 0.2673 - val_acc: 0.9040
Epoch 7/12
60000/60000 [==============================] - 299s 5ms/step - loss: 0.2603 - acc: 0.9070 - val_loss: 0.2538 - 

In [16]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.233760581553
Test accuracy: 0.9168


★ 学習の経過を表す損失関数と精度のグラフは

In [17]:
dlt.utils.plot_loss_and_accuracy(fit,  #model.fitのインスタンス
                                   fname='loss_and_accuracy_graph.png' #保存するファイル名とパス
                                  )

<img src="loss_and_accuracy_graph.png">

とすれば得られます。

In [18]:
# predicted probabilities for the test set
preds = model.predict(X_test)
cls = model.predict_classes(X_test)

#### ★テスト画像に対する分類精度

また分類タスクでは、各テスト画像に対してどの程度の精度で分類されたかを知りたいことが多いですね（というか知らないとだめです）。その時どう出力すればいいか迷うことがあるのですが、dltでは以下のようにすれば、わかりやすい結果が得られます。

In [19]:
# とりあえず10枚
for i in range(10):
    dlt.utils.plot_prediction(
        Yp=preds[i], # 各クラスに対して予測されたラベル 
        X=data.test_images[i], # 各クラスを表す画像
        y=data.test_labels[i], # 正しいクラスのラベル
        classes=data.classes, # ラベル名
        top_n=False, # 上位いくつまで表示させるか. Falseならすべてのカテゴリーに対する精度を表示
        fname='test-%i.png' % i) # 保存するファイル名

<img src="test-0.png">
<img src="test-1.png">
<img src="test-2.png">
<img src="test-3.png">
<img src="test-4.png">

オレンジ色の棒は正しいラベルの分類精度を表し、青色の棒は間違ったラベルの分類精度を表しています。

最後の画像を例とすると、これは`Shirt`とほとんど90%以上の精度で分類していますが、5%程度で`T-shirt/top`と分類しています。

結果を全体的に見たいときは、以下のConfusion Matrixが便利です。

####  ★Confusion Matrix

confusion matrixについては以下のように出力できます。

In [20]:
dlt.utils.plot_confusion_matrix(test_labels=data.test_labels, # 正しいテストラベル(one-hot vectorに変換する前)
                                  y_pred=cls, # np.argmaxを通したYp 
                                  classes=data.classes, # クラス名
                                  title='confusion matrix', # 出力グラフのタイトル
                                  fname='confusion_matrix.png') # 出力パス

<img src="confusion_matrix.png">

縦軸は正しいラベル、横軸は学習によって予測されたラベルを表しています。横軸から見れば、たとえば`Shirt`ラベルを見ると、67.80%で正しく`shirt`と分類されていますが、6.50%で`T-shirt/top`と分類されていることがわかります。

## 宿題

とりあえず、dltに触れるために簡単な宿題を出しておきます。これをこなせばありがたみがわかると思うので・・・

1. MNISTデータセットの場合、上の手順に従えば、どのような結果が得られるでしょうか？（ネットワーク構造は変更しなくても構いません。)

2. 出力画像の保存ディレクトリを変更してみましょう。